In [1]:
from dataclasses import dataclass
from typing import List
import pandas as pd

from typing import Dict, Tuple
from helper.config import TrainConfig
from helper.global_variables import TRAIN_YAML_PATH, EXPERIMENT_YAML_PATH
from helper.models import get_model, get_model_size
from helper.config import ExperimentConfig

# Get Submodel Sizes
from helper.models import get_sub_model 

TARGET_PATH = "models/sub-model-sizes.csv"

In [2]:
def eval_model_size(
    model_name: str,
    layer_config: int,
    dataset_name: str
) -> Dict:
    """Evaluate a single model and return results."""
    # Load and prepare model
    model = get_model(model_name)
    sub_model = get_sub_model(model, model_name, layer_config)

    model_size = get_model_size(sub_model)

    
    return {
        "Model": model_name,
        "Layer Config": layer_config,
        "Dataset": dataset_name,
        "Parameters": model_size,
    }
    
@dataclass
class ModelSizeResult:
    model_name: str
    layer_config: int
    dataset: str
    parameters: int

def evaluate_model_sizes(model_configs: List[Tuple[str, int]]) -> pd.DataFrame:
    """Evaluate sizes of model configurations and return results as DataFrame
    
    Args:
        model_configs: List of (model_name, layer_config) tuples
        
    Returns:
        DataFrame with model size information
    """
    results: List[ModelSizeResult] = []
    
    for model_name, layer_config in model_configs:
        # Parse model info
        dataset_name = model_name.split("-")[0]
        
        # Evaluate size
        result = eval_model_size(
            model_name=model_name,
            layer_config=layer_config,
            dataset_name=dataset_name
        )
        results.append(result)
    
    # Create and format DataFrame
    df = pd.DataFrame(results)
    print(df)
    df.sort_values(by=["Model", "Layer Config"], inplace=True)
    
    return df

In [3]:
# Preparation: Read from Configurations
train_config: TrainConfig = TrainConfig(TRAIN_YAML_PATH)
exp_config: ExperimentConfig = ExperimentConfig(
    EXPERIMENT_YAML_PATH, train_config)

# Main
model_configs = exp_config.get_relevant_model_layer_configs()
print(model_configs)
df_sizes = evaluate_model_sizes(model_configs)
df_sizes.to_csv(TARGET_PATH, index=False)
print("\nEvaluation Results:")
print(df_sizes)

[('cifar10-mobilenet', 3), ('mnist-conv', 1), ('cifar10-squeezenet', 1), ('mnist-dense10x100', 1), ('fmnist-dense', 1), ('mnist-dense', 1), ('cifar10-mobilenet', 2), ('cifar10-squeezenet', 3), ('mnist-dense8x20', 2), ('fmnist-conv', 1), ('cifar10-mobilenet', 1), ('cifar10-squeezenet', 2), ('mnist-dense10x100', 2), ('mnist-dense8x20', 1)]


/home/d.rashedi/miniconda3/envs/py12/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/d.rashedi/miniconda3/envs/py12/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/d.rashedi/miniconda3/envs/py12/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weig

                 Model  Layer Config  Dataset  Parameters
0    cifar10-mobilenet             3  cifar10     2542856
1           mnist-conv             1    mnist        2572
2   cifar10-squeezenet             1  cifar10        1792
3    mnist-dense10x100             1    mnist      129000
4         fmnist-dense             1   fmnist      566144
5          mnist-dense             1    mnist      566144
6    cifar10-mobilenet             2  cifar10      925856
7   cifar10-squeezenet             3  cifar10     1235496
8      mnist-dense8x20             2    mnist       19060
9          fmnist-conv             1   fmnist        2572
10   cifar10-mobilenet             1  cifar10         464
11  cifar10-squeezenet             2  cifar10      558144
12   mnist-dense10x100             2    mnist      179500
13     mnist-dense8x20             1    mnist       17380

Evaluation Results:
                 Model  Layer Config  Dataset  Parameters
10   cifar10-mobilenet             1  cifar10      